In [3]:
import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
from selenium.webdriver.common.by import By
from textblob import TextBlob
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException
import quandl
import requests
import time

In [4]:
API_KEY = "G4xyNzxhrmq82gLxfES5"

In [5]:
snp = pd.read_csv("^GSPC.csv")
nasdaq = pd.read_csv("^IXIC.csv")
nyse = pd.read_csv("^NYA.csv")

In [6]:
def perc_change(row):
    return (row["Close"] - row["Open"])/row["Open"]

nyse["Change"] = nyse.apply(perc_change, axis=1)
nasdaq["Change"] = nasdaq.apply(perc_change, axis=1)
snp["Change"] = snp.apply(perc_change, axis=1)

In [7]:
# snp[snp["Date"] == "2014-10-20"]
nyse

,Date,Open,High,Low,Close,Adj Close,Volume,Change
0,2013-07-10,9340.730469,9372.639648,9314.410156,9340.679688,9340.679688,3011010000,-0.000005
1,2013-07-11,9446.940430,9500.259766,9443.049805,9493.259766,9493.259766,3446340000,0.004903
2,2013-07-12,9488.799805,9498.500000,9465.580078,9498.500000,9498.500000,3039070000,0.001022
3,2013-07-15,9506.459961,9531.660156,9496.919922,9520.950195,9520.950195,2623200000,0.001524
4,2013-07-16,9521.200195,9525.040039,9462.559570,9489.000000,9489.000000,3081710000,-0.003382
5,2013-07-17,9516.580078,9545.250000,9506.089844,9523.790039,9523.790039,3153440000,0.000758
6,2013-07-18,9537.769531,9601.080078,9537.769531,9587.200195,9587.200195,3452370000,0.005183
7,2013-07-19,9587.490234,9618.500000,9565.919922,9618.500000,9618.500000,3302580000,0.003234
8,2013-07-22,9626.009766,9658.080078,9621.969727,9650.580078,9650.580078,2779130000,0.002552
9,2013-07-23,9672.299805,9676.940430,9644.129883,9659.629883,9659.629883,3096180000,-0.001310


In [9]:
dlydata = pd.read_csv("dlydata.csv")
dlydata.rename(columns={'A': 'Symbol', '2014-10-20': 'Date', '52.0700': 'Starting', '52.3800': 'High', '50.8600': 'Low',
             '52.3500': 'Closing', '4942200': 'Volume', '201443': 'Week'}, inplace=True)

In [10]:
dlydata[dlydata['Symbol']=='Z']

,Symbol,Date,Starting,High,Low,Closing,Volume,Week
1034481,Z,2014-10-20,104.66,108.66,104.3100,107.80,1045700,201443
1034482,Z,2014-10-21,109.10,110.40,107.6200,109.01,893600,201443
1034483,Z,2014-10-22,108.62,110.34,103.5400,104.43,1104400,201443
1034484,Z,2014-10-23,105.51,110.00,104.5000,108.45,1287200,201443
1034485,Z,2014-10-24,108.08,108.13,104.8000,106.07,849800,201443
1034486,Z,2014-10-27,105.75,107.83,104.7100,105.55,633700,201444
1034487,Z,2014-10-28,106.06,108.66,104.7200,107.18,1051400,201444
1034488,Z,2014-10-29,107.02,107.02,104.2500,105.52,967300,201444
1034489,Z,2014-10-30,100.50,105.65,98.6000,104.24,1674400,201444
1034490,Z,2014-10-31,106.80,109.66,105.4300,108.73,1373100,201444


In [9]:
def find_nearest_date(future_str_date, date_list):
    while (future_str_date not in date_list.values):
        next_date = datetime.strptime(future_str_date,"%Y-%m-%d")
        future_date = next_date + timedelta(days = 1)
        future_str_date = future_date.strftime('%Y-%m-%d')
    index = list(date_list.values).index(future_str_date)
    return future_str_date, index

In [134]:
def calculate_averages(symbol,dlydata, num_days):
    data = dlydata[dlydata['Symbol']==symbol]
    date_list = data['Date']
    max_date = datetime.strptime(date_list.iloc[len(date_list)-1],"%Y-%m-%d")
    moving_averages = {}
    sum_averages = {}
    prev_end = None
    prev_iter_date = None
    for i in range(0, len(date_list)):
        if i == 0:
            str_date = date_list.iloc[i]
            this_date = datetime.strptime(str_date,"%Y-%m-%d")
            future_date = this_date + timedelta(days=num_days)
            future_str_date = future_date.strftime('%Y-%m-%d')
            if future_date + timedelta(3) <= max_date:
                future_str_date, index = find_nearest_date(future_str_date, date_list)
                sum_avg = 0
                for j in range(i,index+1):
                    sum_avg += data[data['Date']==date_list.iloc[j]].iloc[0]['Closing']
                moving_averages[future_str_date] = sum_avg/(index + 1 - i)
                sum_averages[future_str_date] = sum_avg
        else:
            curr_date = date_list.iloc[i]
            prev_date = date_list.iloc[i-1]
            time_diff = (datetime.strptime(curr_date, "%Y-%m-%d") - datetime.strptime(prev_date, "%Y-%m-%d")).days
            future_date = datetime.strptime(prev_iter_date, "%Y-%m-%d") + timedelta(time_diff)
            future_str_date = future_date.strftime('%Y-%m-%d')
            if future_date + timedelta(3) <= max_date:
                future_str_date, index = find_nearest_date(future_str_date, date_list)
                add = 0
                for j in range(prev_end, index+1):
                    add += data[data['Date']==date_list.iloc[j]].iloc[0]['Closing']
                subtract = data[data['Date']==date_list.iloc[prev_end-1]].iloc[0]['Closing']
                this_sum = sum_averages[prev_iter_date] - subtract + add
                sum_averages[future_str_date] = this_sum
                moving_averages[future_str_date] = this_sum/(index - i)
        prev_iter_date = future_str_date
        prev_end = index+1
    return moving_averages


In [11]:
def percent_positive(percent_change):
    if len(percent_change)==0:
        return 0
    return len([percent_change[i] for i in percent_change.keys() if percent_change[i]>0])/len(percent_change)

In [12]:
def create_dict(data):
    dict = {}
    for i,row in data.iterrows():
        dict[row["Date"]] = row["Closing"]
    return dict

In [132]:
def populate_df(dict, averages, data, num_days):
    new_averages = []
    for key in dict:
        if key in list(averages.keys()):
            new_averages.append(averages[key])
        else:
            new_averages.append(None)
    data[str(num_days) + " Day Moving Average"] = new_averages
    return data

In [14]:
def add_market_performance(data, snp, nyse, nasdaq):    
    def format_snp(date, date_list, dset):
        this_date = date
        first = this_date[0:6]
        second = this_date[6:]
        d1 = first.index('-')
        d2 = second.index('-') + 6
        mid = this_date[d1+1:d2]
        end = this_date[d2+1:]
        if mid[0] == '0':
            mid = mid[1:]
        if end[0] == '0':
            end = end[1:]
        formatted = mid + "/" + end + "/" + this_date[0:d1]
        if formatted in date_list:
#             print (True)
#             print(dset[dset["Date"] == date].iloc[0]["Change"])
            return dset[dset["Date"] == formatted].iloc[0]["Change"]
        else:
            return None
    def format_other(date,date_list, dset):
        if date in date_list: 
            return dset[dset["Date"] == date].iloc[0]["Change"]
        else:
            return None
    def snp_change(row):
        return format_snp(row["Date"], snp["Date"].values, snp)
    def nyse_change(row):
        return format_other(row["Date"],nyse["Date"].values, nyse)
    def nasdaq_change(row):
        return format_other(row["Date"],nasdaq["Date"].values, nasdaq)
    
    data["snp_change"] = data.apply(snp_change,axis=1)
    data["nyse_change"] = data.apply(nyse_change,axis=1)
    data["nasdaq_change"] = data.apply(nasdaq_change,axis=1)
    
#     print(data["snp_change"])
    
    return data
    

In [26]:
def convertToDate(datetext):
    week_days = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
    abbr_to_num = {"Jan":"01","Feb":"02","Mar":"03","Apr":"04","May":"05","Jun":"06", "Jul":"07","Aug":"08",
                   "Sep": "09","Oct": "10","Nov": "11","Dec": "12"}
    year = None
    month = None
    day = None
    if datetext[0:5] == "Today":
        d = datetime.today()
        return d.strftime('%Y-%m-%d')
    if datetext[0:9] == "Yesterday":
        d = datetime.today()
        yest = d - timedelta(days=1)
        return yest.strftime('%Y-%m-%d')
    elif datetext[0:3] in week_days:
        year = datetime.now().strftime("%Y")
        comma_index_2 = datetext[5:].index(",") + 5
        month = abbr_to_num[datetext[5:8]]
        day = datetext[10:comma_index_2]
        if month == "05":
            day = datetext[9:comma_index_2]
    else:
        comma_index_1 = datetext.index(",")
        year = datetext[comma_index_1:][2:6]
        month = abbr_to_num[datetext[0:3]]
        day = datetext[5:comma_index_1]
        if month == "05":
            day = datetext[4:comma_index_1]
    corr_date = year + "-" + month + "-" + day
    return corr_date

def date_conversion(datetext):
    abbr_to_num = {"Jan":"01","Feb":"02","Mar":"03","Apr":"04","May":"05","Jun":"06", "Jul":"07","Aug":"08",
                   "Sep": "09","Oct": "10","Nov": "11","Dec": "12"}
    comma_index_1 = datetext.index(",")
    year = datetext[comma_index_1:][2:6]
    month = abbr_to_num[datetext[0:3]]
    day = datetext[4:comma_index_1]
    corr_date = year + "-" + month + "-" + day
    return corr_date 

date_conversion("Aug 06, 2019")


'2019-08-06'

In [27]:
def getSentiment(ticker, start_date):
    lower_ticker = ticker.lower()
    date_dict = {}
    url = 'https://seekingalpha.com/symbol/' + ticker + '?s=' + lower_ticker + '&news=news-all'
    source = requests.get(url)

    driver = webdriver.Edge(executable_path="C:/Users/Adhithya/Desktop/MicrosoftDriver/MicrosoftWebDriver.exe")
    driver.get(url)
    # html = driver.execute_script("return document.documentElement.innerHTML")

#     driver.maximize_window()
    d_obj = datetime.today()
    print(d_obj)
    vars = None
    while d_obj > start_date:
        vars = driver.find_elements_by_class_name("mc_list_li")
        date = vars[len(vars)-1].find_element_by_class_name("date")
        corr_date = convertToDate(date.text)
        d_obj = datetime.strptime(corr_date, '%Y-%m-%d')
        driver.execute_script("window.scrollBy(0,document.body.scrollHeight)")

        
    for var in vars:
        # driver.execute_script("arguments[0].scrollIntoView();", var)
        list_items = var.find_elements_by_tag_name("li")
        avg_polarity = 0
        for l_item in list_items:
            analysis = TextBlob(l_item.text)
            avg_polarity += analysis.sentiment.polarity
        if len(list_items) == 0:
            avg_polarity = None
        else:
            avg_polarity = avg_polarity/len(list_items)
        date = var.find_element_by_class_name("date")
        corr_date = convertToDate(date.text)
        d_obj = datetime.strptime(corr_date,'%Y-%m-%d')
        date_dict[corr_date] = avg_polarity
        
    driver.quit()
    return date_dict

In [28]:
def add_sentiment(stock_df, sentiment_dict):
    
    def sentiment(date):
        if date not in sentiment_dict.keys():
            return None
        return sentiment_dict[date]
    stock_df["Sentiment"] = stock_df["Date"].apply(sentiment)
    return stock_df

In [18]:
def add_sector_performance(url, sector, sector_dict):
    stock_list = []
    driver = webdriver.Edge(executable_path="C:/Users/Adhithya/Desktop/MicrosoftDriver/MicrosoftWebDriver.exe")
    offset = 0
    ind = url.index("sector")
    prefix = url[0:ind]
    sector_part = url[(ind + 7):]
    url_base = prefix + "screener/predefined/" + sector_part
    while True:
        try:
#             url = sector_url + "&count=25&offset=" + str(offset)
            print(url)
            driver.get(url)
            print(offset)
            mytable = driver.find_element_by_css_selector("#scr-res-table > div.Ovx\(a\).Ovx\(h\)--print.Ovy\(h\) > table")
            rows = mytable.find_elements_by_tag_name("tr")
            offset += len(rows[1:])
            for row in rows[1:]:
                cells = row.find_elements_by_tag_name("td")
                ticker = cells[0].text
                new_line_index = ticker.index('\n')
                stock_list.append(ticker[0:new_line_index])
#                 print(cells[0].text)
            if len(rows) < 25:
                break
            if offset == 25:
                url = url_base + "?offset=25&count=25"
            else:
                url = url_base + "?offset=" + str(offset) + "&count=25"
        except Exception as e:
            print("exception")
            driver.quit()
#             if isinstance(e, NoSuchElementException):    
#                 print("no such element")
#                 url = sector_url + "?count=25&offset=" + str(offset)
#             else:
            url = url_base + "?offset=" + str(offset) + "&count=25"
            driver = webdriver.Edge(executable_path="C:/Users/Adhithya/Desktop/MicrosoftDriver/MicrosoftWebDriver.exe")
            driver.get(url)
    sector_dict[sector] = stock_list      
    driver.quit()
    return sector_dict
        

# https://finance.yahoo.com/screener/predefined/ms_utilities?offset=25&count=25

In [19]:
sector_urls = {}
sector_urls["Basic Materials"] = "https://finance.yahoo.com/sector/ms_basic_materials"
sector_urls["Communication Services"] = "https://finance.yahoo.com/sector/ms_communication_services"
sector_urls["Consumer Cyclical"] = "https://finance.yahoo.com/sector/ms_consumer_cyclical"
sector_urls["Consumer Defensive"] = "https://finance.yahoo.com/sector/ms_consumer_defensive"
sector_urls["Energy"] = "https://finance.yahoo.com/sector/ms_energy"
sector_urls["Financial Services"] = "https://finance.yahoo.com/sector/ms_financial_services"
sector_urls["Healthcare"] = "https://finance.yahoo.com/sector/ms_healthcare"
sector_urls["Industrials"] = "https://finance.yahoo.com/sector/ms_industrials"
sector_urls["Real Estate"] = "https://finance.yahoo.com/sector/ms_real_estate"
sector_urls["Technology"] = "https://finance.yahoo.com/sector/ms_technology"
sector_urls["Utilities"] = "https://finance.yahoo.com/sector/ms_utilities"

In [54]:
def populate_sector_stocks():
    sector_dictionary = {}
    for sector in sector_urls.keys():
        print("DONE")
        url = sector_urls[sector]
        sector_dictionary = add_sector_performance(url, sector, sector_dictionary)
    return sector_dictionary

In [55]:
sector_dictionary = populate_sector_stocks()

DONE
https://finance.yahoo.com/sector/ms_basic_materials
0
https://finance.yahoo.com/screener/predefined/ms_basic_materials?offset=25&count=25
25
https://finance.yahoo.com/screener/predefined/ms_basic_materials?offset=50&count=25
50
https://finance.yahoo.com/screener/predefined/ms_basic_materials?offset=75&count=25
75
https://finance.yahoo.com/screener/predefined/ms_basic_materials?offset=100&count=25
100
https://finance.yahoo.com/screener/predefined/ms_basic_materials?offset=125&count=25
125
https://finance.yahoo.com/screener/predefined/ms_basic_materials?offset=150&count=25
150
https://finance.yahoo.com/screener/predefined/ms_basic_materials?offset=175&count=25
175
DONE
https://finance.yahoo.com/sector/ms_communication_services
0
https://finance.yahoo.com/screener/predefined/ms_communication_services?offset=25&count=25
25
https://finance.yahoo.com/screener/predefined/ms_communication_services?offset=50&count=25
50
DONE
https://finance.yahoo.com/sector/ms_consumer_cyclical
0
https://f

50
https://finance.yahoo.com/screener/predefined/ms_healthcare?offset=75&count=25
75
https://finance.yahoo.com/screener/predefined/ms_healthcare?offset=100&count=25
100
https://finance.yahoo.com/screener/predefined/ms_healthcare?offset=125&count=25
125
https://finance.yahoo.com/screener/predefined/ms_healthcare?offset=150&count=25
150
https://finance.yahoo.com/screener/predefined/ms_healthcare?offset=175&count=25
175
https://finance.yahoo.com/screener/predefined/ms_healthcare?offset=200&count=25
200
https://finance.yahoo.com/screener/predefined/ms_healthcare?offset=225&count=25
225
https://finance.yahoo.com/screener/predefined/ms_healthcare?offset=250&count=25
250
https://finance.yahoo.com/screener/predefined/ms_healthcare?offset=275&count=25
275
https://finance.yahoo.com/screener/predefined/ms_healthcare?offset=300&count=25
300
https://finance.yahoo.com/screener/predefined/ms_healthcare?offset=325&count=25
325
DONE
https://finance.yahoo.com/sector/ms_industrials
0
https://finance.yaho

In [56]:
saved_sector_dict = sector_dictionary

In [57]:
saved_sector_dict

{'Basic Materials': ['BHP',
  'BBL',
  'LIN',
  'CTA-PB',
  'RIO',
  'CTA-PA',
  'ECL',
  'VALE',
  'APD',
  'DD',
  'SHW',
  'DOW',
  'GOLD',
  'NEM',
  'NTR',
  'PPG',
  'CRH',
  'LYB',
  'SCCO',
  'CTVA',
  'VMC',
  'FNV',
  'MLM',
  'PKX',
  'NUE',
  'AEM',
  'FCX',
  'CE',
  'MT',
  'IFF',
  'WPM',
  'MAS',
  'FMC',
  'SUZ',
  'CF',
  'JHX',
  'KL',
  'TECK',
  'AVTR-PA',
  'AVTR',
  'EMN',
  'RPM',
  'AU',
  'RGLD',
  'WLK',
  'MOS',
  'VEDL',
  'ACH',
  'SQM',
  'ALB',
  'AXTA',
  'RS',
  'SMG',
  'STLD',
  'OC',
  'VSM',
  'MDU',
  'NEU',
  'TREX',
  'GFI',
  'AWI',
  'HUN',
  'ASH',
  'GRA',
  'BVN',
  'PAAS',
  'EXP',
  'RYN',
  'TX',
  'NGVT',
  'AA',
  'CCJ',
  'UNVR',
  'BLD',
  'BCPC',
  'KWR',
  'LPX',
  'SXT',
  'OLN',
  'AGI',
  'WDFC',
  'MEOH',
  'UFPI',
  'PVG',
  'ESI',
  'AAON',
  'POL',
  'CLF',
  'FUL',
  'CBT',
  'BLDR',
  'UFS',
  'SCL',
  'BECN',
  'SUM',
  'AG',
  'IOSP',
  'WOR',
  'X',
  'CC',
  'PQG',
  'SSRM',
  'BTU',
  'ARLP',
  'CMC',
  'OSB',
  'JELD

In [107]:
def get_sector_day_averages(averages_dict, stock_list):
    curr_stock_dict = {}
    lowest_date = datetime.strptime("2014-01-01","%Y-%m-%d")
    
    api_key = "S52ZB5D8LU3M5FN0"
    
    for ticker in stock_list:
        print(ticker)
        print(key)
        API_URL = "https://www.alphavantage.co/query"
        
        data = {
            "function": "TIME_SERIES_DAILY",
            "symbol": ticker,
            "outputsize": "full",
            "datatype": "json",
            "apikey": api_key
            }

        response = requests.get(API_URL, data)
        json = response.json()

        if "Error Message" not in json.keys():
            json_data = json["Time Series (Daily)"]
            for date in json_data.keys():
                this_date = datetime.strptime(date,"%Y-%m-%d")
                if this_date > lowest_date:
                    price_data = json_data[date]
                    open = float(price_data['1. open'])
                    close = float(price_data['4. close'])
                    perc_change = (close - open)/open
                    if date in averages_dict.keys():
                        existing = averages_dict[date]
                        existing.append(perc_change)
                        averages_dict[date] = existing
                    else:
                        averages_dict[date] = [perc_change]
                    
    return averages_dict
                
                
# get_sector_day_averages(['CTA-PA'])

In [106]:
sector_day_averages = {}
curr_index = 0
count = 0
key_index = 0
key = api_keys_list[key_index]
num_keys = len(api_keys_dict.keys())
for sector in sector_dictionary:
    print(sector)
    this_dict = {}
    sector_stocks = sector_dictionary[sector]
    symbol_list = random.sample(list(sector_stocks), 40)
    i = 0
    while i < len(symbol_list):
        sector_subgroup = symbol_list[i:i+5]
        this_dict = get_sector_day_averages(this_dict, sector_subgroup)
        i += 5
        time.sleep(60)
    sector_day_averages[sector] = this_dict
    

Basic Materials
OEC
G1B0U3OOHJIA9YUZ
AVD
G1B0U3OOHJIA9YUZ
GOLD
G1B0U3OOHJIA9YUZ
BLDR
G1B0U3OOHJIA9YUZ
BTU
G1B0U3OOHJIA9YUZ
DOOR
G1B0U3OOHJIA9YUZ
USLM
G1B0U3OOHJIA9YUZ
CLW
G1B0U3OOHJIA9YUZ
CBT
G1B0U3OOHJIA9YUZ
CC
G1B0U3OOHJIA9YUZ
ESI
G1B0U3OOHJIA9YUZ
OSB
G1B0U3OOHJIA9YUZ
RGLD
G1B0U3OOHJIA9YUZ
ROCK
G1B0U3OOHJIA9YUZ
AVTR
G1B0U3OOHJIA9YUZ
ACH
G1B0U3OOHJIA9YUZ
MEOH
G1B0U3OOHJIA9YUZ
ASH
G1B0U3OOHJIA9YUZ
NEM
G1B0U3OOHJIA9YUZ
AWI
G1B0U3OOHJIA9YUZ
BECN
G1B0U3OOHJIA9YUZ
UFPI
G1B0U3OOHJIA9YUZ
LOMA
G1B0U3OOHJIA9YUZ
ECL
G1B0U3OOHJIA9YUZ
BVN
G1B0U3OOHJIA9YUZ
KRA
G1B0U3OOHJIA9YUZ
AVTR-PA
G1B0U3OOHJIA9YUZ
KL
G1B0U3OOHJIA9YUZ
BCC
G1B0U3OOHJIA9YUZ
NX
G1B0U3OOHJIA9YUZ
MDU
G1B0U3OOHJIA9YUZ
HUN
G1B0U3OOHJIA9YUZ
MLM
G1B0U3OOHJIA9YUZ
DD
G1B0U3OOHJIA9YUZ
UFS
G1B0U3OOHJIA9YUZ
PVG
G1B0U3OOHJIA9YUZ
SHW
G1B0U3OOHJIA9YUZ
NC
G1B0U3OOHJIA9YUZ
BXC
G1B0U3OOHJIA9YUZ
PQG
G1B0U3OOHJIA9YUZ
Communication Services
TEF
G1B0U3OOHJIA9YUZ
CMCSA
G1B0U3OOHJIA9YUZ
WOW
G1B0U3OOHJIA9YUZ
TU
G1B0U3OOHJIA9YUZ
TSU
G1B0U3OOHJIA9YUZ
CCOI


SMTC
G1B0U3OOHJIA9YUZ
DAKT
G1B0U3OOHJIA9YUZ
CRUS
G1B0U3OOHJIA9YUZ
JBL
G1B0U3OOHJIA9YUZ
ADSK
G1B0U3OOHJIA9YUZ
TENB
G1B0U3OOHJIA9YUZ
VEEV
G1B0U3OOHJIA9YUZ
CVET
G1B0U3OOHJIA9YUZ
SOHU
G1B0U3OOHJIA9YUZ
SMAR
G1B0U3OOHJIA9YUZ
SE
G1B0U3OOHJIA9YUZ
TNAV
G1B0U3OOHJIA9YUZ
NSIT
G1B0U3OOHJIA9YUZ
CISN
G1B0U3OOHJIA9YUZ
UEIC
G1B0U3OOHJIA9YUZ
CDAY
G1B0U3OOHJIA9YUZ
ALLT
G1B0U3OOHJIA9YUZ
SEDG
G1B0U3OOHJIA9YUZ
SCWX
G1B0U3OOHJIA9YUZ
TZOO
G1B0U3OOHJIA9YUZ
IRBT
G1B0U3OOHJIA9YUZ
GWRE
G1B0U3OOHJIA9YUZ
TDC
G1B0U3OOHJIA9YUZ
DXC
G1B0U3OOHJIA9YUZ
LFUS
G1B0U3OOHJIA9YUZ
RMBS
G1B0U3OOHJIA9YUZ
STM
G1B0U3OOHJIA9YUZ
QTWO
G1B0U3OOHJIA9YUZ
FFIV
G1B0U3OOHJIA9YUZ
INFY
G1B0U3OOHJIA9YUZ
Utilities
AGR
G1B0U3OOHJIA9YUZ
KEN
G1B0U3OOHJIA9YUZ
NWN
G1B0U3OOHJIA9YUZ
NI-PB
G1B0U3OOHJIA9YUZ
IPLDP
G1B0U3OOHJIA9YUZ
CWEN
G1B0U3OOHJIA9YUZ
IDA
G1B0U3OOHJIA9YUZ
AWR
G1B0U3OOHJIA9YUZ
DTE
G1B0U3OOHJIA9YUZ
EDN
G1B0U3OOHJIA9YUZ
ATO
G1B0U3OOHJIA9YUZ
UTL
G1B0U3OOHJIA9YUZ
CPK
G1B0U3OOHJIA9YUZ
EBR
G1B0U3OOHJIA9YUZ
CTWS
G1B0U3OOHJIA9YUZ
TERP
G1B0U3OOHJ

In [108]:
saved_sector_day_averages = sector_day_averages

In [110]:
new_sector_averages = {}
for sector in sector_day_averages:
    sector_dict = sector_day_averages[sector]
    this_dict = {}
    for date in sector_dict:
        day_values = sector_dict[date]
        mean = np.mean(day_values)
        this_dict[date] = mean
    new_sector_averages[sector] = this_dict


In [113]:
sector_day_averages = new_sector_averages

In [115]:
def calculations(data,moving_averages,dict):
    three_month = {}
    six_month = {}
    twelve_month = {}
    date_list = data['Date']
    max_date = datetime.strptime(list(moving_averages.keys())[len(moving_averages.keys())-1],'%Y-%m-%d')
    for key in list(moving_averages.keys()):
        key_date = datetime.strptime(key,'%Y-%m-%d')
        curr_price = dict[key]
        plus_three = key_date + timedelta(days=90)
        plus_three_str = plus_three.strftime('%Y-%m-%d')
        plus_six = key_date + timedelta(days=180)
        plus_six_str = plus_six.strftime('%Y-%m-%d')
        plus_twelve = key_date + timedelta(days=360)
        plus_twelve_str = plus_twelve.strftime('%Y-%m-%d')
        if plus_three < max_date:
            plus_three_str, index = find_nearest_date(plus_three_str, date_list)
            plus_three_price = dict[plus_three_str]
            three_percent_change = (plus_three_price - curr_price)/curr_price
        if plus_six < max_date:
            plus_six_str, index = find_nearest_date(plus_six_str, date_list)
            plus_six_price = dict[plus_six_str]
            six_percent_change = (plus_six_price - curr_price)/curr_price
        if plus_twelve < max_date:
            plus_twelve_str, index = find_nearest_date(plus_twelve_str, date_list)
            plus_twelve_price = dict[plus_twelve_str]
            twelve_percent_change = (plus_twelve_price - curr_price)/curr_price
        if curr_price > moving_averages[key]:
            three_month[key] = three_percent_change
            six_month[key] = six_percent_change
            twelve_month[key] = twelve_percent_change
    return three_month, six_month, twelve_month

In [210]:
def add_sectors(stock_df, sector_dictionary, sector_averages):
            
    def sector_name(df):
        for sector in sector_dictionary.keys():
            if df["Symbol"] in sector_dictionary[sector]:
                return sector
        return None
    
    def sector_data(df):
        symbol = df["Symbol"]
        date = df["Date"]
        symbol_sector = None
        for sector in sector_dictionary.keys():
            if symbol in sector_dictionary[sector]:
                symbol_sector = sector
                
        if symbol_sector is None:
            return None

        day_data = sector_averages[symbol_sector]
        if date not in day_data.keys():
            return None
        return day_data[date]
    
    stock_df["Sector"] = stock_df.apply(sector_name, axis=1)
    stock_df["Sector Change"] = stock_df.apply(sector_data, axis=1)
    return stock_df

In [15]:
stocks = dlydata['Symbol'].unique()
# symbol_list = random.sample(list(stocks), 3)
symbol_list = stocks[0:int(len(stocks)/2)]

In [16]:
stock_df = pd.DataFrame(columns=['Symbol', 'Date', 'Starting', 'High', 'Low', 'Closing', 'Volume', 'Week','snp_change',
                                 'nyse_change','nasdaq_change'])

In [17]:
symbol_list

array(['A', 'AA', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT',
       'ACAD', 'ACC', 'ACGL', 'ACHC', 'ACM', 'ACN', 'ADBE', 'ADI', 'ADM',
       'ADNT', 'ADP', 'ADS', 'ADSK', 'AEE', 'AEP', 'AES', 'AET', 'AFG',
       'AFL', 'AGCO', 'AGIO', 'AGN', 'AGNC', 'AGO', 'AGR', 'AHL', 'AIG',
       'AIV', 'AIZ', 'AJG', 'AKAM', 'AKRX', 'AL', 'ALB', 'ALGN', 'ALK',
       'ALKS', 'ALL', 'ALLE', 'ALLY', 'ALNY', 'ALR', 'ALSN', 'ALXN',
       'AMAT', 'AMCX', 'AMD', 'AME', 'AMG', 'AMGN', 'AMH', 'AMP', 'AMT',
       'AMTD', 'AMZN', 'AN', 'ANAT', 'ANET', 'ANSS', 'ANTM', 'AON', 'AOS',
       'APA', 'APC', 'APD', 'APH', 'APLE', 'AR', 'ARD', 'ARE', 'ARMK',
       'ARNC', 'ARRS', 'ARW', 'ASB', 'ASH', 'ATH', 'ATHN', 'ATO', 'ATR',
       'ATVI', 'AVB', 'AVGO', 'AVT', 'AVY', 'AWH', 'AWI', 'AWK', 'AXP',
       'AXS', 'AXTA', 'AYI', 'AZO', 'BA', 'BAC', 'BAH', 'BAX', 'BBBY',
       'BBT', 'BBY', 'BC', 'BCR', 'BDN', 'BDX', 'BEN', 'BERY', 'BF.A',
       'BF.B', 'BFAM', 'BG', 'BGCP', 'BHI', 'BIIB', 'BIO', 'BIVV

In [204]:
three_list = {}
six_list = {}
twelve_list = {}
for symbol in symbol_list:
    print(symbol)
    symbol_data = dlydata[dlydata['Symbol']==symbol]
    first_date = symbol_data.iloc[0]["Date"]
#     print(first_date)
    dict = create_dict(symbol_data)
    averages200 = calculate_averages(symbol, dlydata, 200)
    averages90 = calculate_averages(symbol, dlydata, 90)
    averages30 = calculate_averages(symbol, dlydata, 30)
    data = populate_df(dict, averages200, symbol_data, 200)
    data = populate_df(dict, averages90, data, 90)
    data = populate_df(dict, averages30, data, 30)
    data = add_market_performance(data,snp, nyse, nasdaq)
    stock_df = stock_df.append(data, sort=False)


#     three_month, six_month, twelve_month = calculations(symbol_data, averages, dict)
#     three_percent, six_percent, twelve_percent = percent_positive(three_month), percent_positive(six_month), percent_positive(twelve_month)
#     three_list[symbol] = three_percent
#     six_list[symbol] = six_percent
#     twelve_list[symbol] = twelve_percent
df = add_sectors(stock_df, symbol, sector_dictionary, sector_day_averages)
df.to_csv("C:/Users/Adhithya/Desktop/output.csv", index=True, header=True)

A


C:\Users\Adhithya\Anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Adhithya\Anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Adhithya\Anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

AA
AAL
AAP
AAPL
ABBV
ABC
ABMD
ABT
ACAD
ACC
ACGL
ACHC
ACM
ACN
ADBE
ADI
ADM
ADNT
ADP
ADS
ADSK
AEE
AEP
AES
AET
AFG
AFL
AGCO
AGIO
AGN
AGNC
AGO
AGR
AHL
AIG
AIV
AIZ
AJG
AKAM
AKRX
AL
ALB
ALGN
ALK
ALKS
ALL
ALLE
ALLY
ALNY
ALR
ALSN
ALXN
AMAT
AMCX
AMD
AME
AMG
AMGN
AMH
AMP
AMT
AMTD
AMZN
AN
ANAT
ANET
ANSS
ANTM
AON
AOS
APA
APC
APD
APH
APLE
AR
ARD
ARE
ARMK
ARNC
ARRS
ARW
ASB
ASH
ATH
ATHN
ATO
ATR
ATVI
AVB
AVGO
AVT
AVY
AWH
AWI
AWK
AXP
AXS
AXTA
AYI
AZO
BA
BAC
BAH
BAX
BBBY
BBT
BBY
BC
BCR
BDN
BDX
BEN
BERY
BF.A
BF.B
BFAM
BG
BGCP
BHI
BIIB
BIO
BIVV
BK
BKD
BKFS
BKU
BLK
BLL
BMRN
BMS
BMY
BOH
BOKF
BPOP
BR
BRCD
BRK.B
BRKR
BRO
BRX
BSX
BUFF
BURL
BWA
BWXT
BXP
C
CA
CAA
CAB
CABO
CACC
CAG
CAH
CASY
CAT
CAVM
CB
CBG
CBOE
CBS
CBSH
CBT
CC
CCI
CCK
CCL
CDEV
CDK
CDNS
CDW
CE
CELG
CERN
CF
CFG
CFR
CFX
CGNX
CHD
CHH
CHK
CHRW
CHTR
CI
CIM
CINF
CIT
CL
CLGX
CLH
CLNS
CLR
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNA
CNC
CNDT
CNK
CNP
CNX
COF
COG
COH
COHR
COL
COMM
CONE
COO
COP
COR
COST
COTY
CPA
CPB
CPN
CPRT
CPT
CR
CRI
CRL
CRM
CSCO
CSGP
CSL
CSRA
CST
CS

NameError: name 'stocks_df' is not defined

In [211]:
df = add_sectors(stock_df, sector_dictionary, sector_day_averages)

In [213]:
df.to_csv("C:/Users/Adhithya/Desktop/sector_output.csv", index=True, header=True)

In [11]:
csv = pd.read_csv("C:/Users/Adhithya/Desktop/sector_output.csv")

In [12]:
csv

,Unnamed: 0,Symbol,Date,Starting,High,Low,Closing,Volume,Week,snp_change,nyse_change,nasdaq_change,200 Day Moving Average,90 Day Moving Average,30 Day Moving Average,Sector,Sector Change
0,0,A,2014-10-21,52.86,54.0300,52.860,54.00,2231800,201443,0.016707,0.012543,0.013835,NaN,NaN,NaN,Healthcare,0.021057
1,1,A,2014-10-22,53.77,53.8600,52.990,53.05,2161000,201443,-0.007304,-0.009170,-0.010456,NaN,NaN,NaN,Healthcare,-0.013826
2,2,A,2014-10-23,53.59,54.2100,53.590,53.71,1754700,201443,0.010254,0.004100,0.005726,NaN,NaN,NaN,Healthcare,0.008753
3,3,A,2014-10-24,53.70,54.1100,53.510,54.05,1508100,201443,0.006656,0.005066,0.005440,NaN,NaN,NaN,Healthcare,0.002980
4,4,A,2014-10-27,53.94,54.1100,53.600,53.99,1105200,201444,-0.000683,-0.000307,0.003784,NaN,NaN,NaN,Healthcare,0.008289
5,5,A,2014-10-28,54.12,54.6700,53.880,54.66,1714300,201444,0.010646,0.009546,0.012997,NaN,NaN,NaN,Healthcare,0.013797
6,6,A,2014-10-29,54.59,54.6800,54.130,54.50,1484200,201444,-0.000499,-0.004083,-0.000470,NaN,NaN,NaN,Healthcare,-0.003975
7,7,A,2014-10-30,54.08,54.3800,53.750,54.32,2062800,201444,0.007659,0.008098,0.007511,NaN,NaN,NaN,Healthcare,0.016956
8,8,A,2014-10-31,55.14,55.4100,54.850,55.28,2087800,201444,0.008420,0.006690,-0.001877,NaN,NaN,NaN,Healthcare,-0.005429
9,9,A,2014-11-03,39.90,41.2400,39.490,40.84,4559900,201445,-0.000198,-0.002375,0.001122,NaN,NaN,NaN,Healthcare,-0.003574
